# On the day Appointments 

- Refresh catalog for GP appointments if needed --> done
- Ensure this does not break existing notebooks/analysis and make changes if required.
- Calculate % on the day (same days) appointments for each month in each ICB Area
- Line plot of last 1+ years % on the day appointments by ICB.
- Bar plot, x values: ICB, y values:% on the day appointments for the last 12 months of appointment data.
- Output Yaml as below:
   07K : 0.3
   .. other ICBs
  outputs/same-day-appointments.yaml

In [19]:
from os import chdir
from pathlib import Path
if 'notebooks' in str(Path.cwd()):
    chdir('..')

# Library imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import category_encoders

# project imports from src
from src.schemas import DataCatalog
from src.various_methods import PlotCounter

## **Loading the catalog**

In [20]:
# Load data catalog
data_catalog = DataCatalog.load_from_yaml("data_catalog.yaml")

# Initializing the plot counter (Keeps track of plot numbers)
plot_counter = PlotCounter(name="Referral_rate") 

### Loading the Referrals dataframe from 2022-23

In [21]:
gp_appts_catalog_entry = data_catalog.single_data_sources[0]
gp_appts_df = gp_appts_catalog_entry.load()
print(gp_appts_catalog_entry.zip_file.namelist())

# Getting the data for SNEE-sub icb areas only
dfs = []
for ccg_name in gp_appts_catalog_entry.zip_file.namelist():
    if ccg_name == "Regional_CSV_SuffolkNEEssex.csv":
        gp_appts_catalog_entry.csv_file = ccg_name
        dfs.append(gp_appts_catalog_entry.load())

snee_appointments_df = pd.concat(dfs, axis=0)
snee_appointments_df.head()

['Regional_CSV_WYorks.csv', 'Regional_CSV_Sussex.csv', 'Regional_CSV_SurreyHearts.csv', 'Regional_CSV_SuffolkNEEssex.csv', 'Regional_CSV_StaffsStoke.csv', 'Regional_CSV_Somerset.csv', 'Regional_CSV_ShropsTelfordWrekin.csv', 'Regional_CSV_SYorks.csv', 'Regional_CSV_SWLondon.csv', 'Regional_CSV_SELondon.csv', 'Regional_CSV_NottinghamNotts.csv', 'Regional_CSV_Northamptonshire.csv', 'Regional_CSV_NorfolkWaveney.csv', 'Regional_CSV_NWLondon.csv', 'Regional_CSV_NENCumbria.csv', 'Regional_CSV_NELondon.csv', 'Regional_CSV_NCLondon.csv', 'Regional_CSV_MidSEssex.csv', 'Regional_CSV_Lincolnshire.csv', 'Regional_CSV_LeicesterLshire.csv', 'Regional_CSV_LancsSCumbria.csv', 'Regional_CSV_KentMedway.csv', 'Regional_CSV_HumberNYorks.csv', 'Regional_CSV_HertfordshireWEssex.csv', 'Regional_CSV_HereWorcs.csv', 'Regional_CSV_HampshireIoW.csv', 'Regional_CSV_Gloucestershire.csv', 'Regional_CSV_GManchesterHSCP.csv', 'Regional_CSV_FrimleyHealth.csv', 'Regional_CSV_Dorset.csv', 'Regional_CSV_Devon.csv', 'Regio

,SUB_ICB_LOCATION_CODE,SUB_ICB_LOCATION_ONS_CODE,SUB_ICB_LOCATION_NAME,ICB_ONS_CODE,ICB_NAME,REGION_ONS_CODE,REGION_NAME,APPOINTMENT_MONTH,APPT_STATUS,HCP_TYPE,APPT_MODE,TIME_BETWEEN_BOOK_AND_APPT,COUNT_OF_APPOINTMENTS
0,06L,E38000086,NHS Suffolk and North East Essex ICB - 06L,E54000023,NHS Suffolk and North East Essex Integrated Ca...,E40000007,East of England,JUL2021,Attended,GP,Face-to-Face,1 Day,3215
1,06L,E38000086,NHS Suffolk and North East Essex ICB - 06L,E54000023,NHS Suffolk and North East Essex Integrated Ca...,E40000007,East of England,JUL2021,Attended,GP,Face-to-Face,15 to 21 Days,705
2,06L,E38000086,NHS Suffolk and North East Essex ICB - 06L,E54000023,NHS Suffolk and North East Essex Integrated Ca...,E40000007,East of England,JUL2021,Attended,GP,Face-to-Face,2 to 7 Days,5045
3,06L,E38000086,NHS Suffolk and North East Essex ICB - 06L,E54000023,NHS Suffolk and North East Essex Integrated Ca...,E40000007,East of England,JUL2021,Attended,GP,Face-to-Face,22 to 28 Days,306
4,06L,E38000086,NHS Suffolk and North East Essex ICB - 06L,E54000023,NHS Suffolk and North East Essex Integrated Ca...,E40000007,East of England,JUL2021,Attended,GP,Face-to-Face,8 to 14 Days,1900


In [25]:
print(snee_appointments_df["TIME_BETWEEN_BOOK_AND_APPT"]).unique()

0                    1 Day
1           15  to 21 Days
2              2 to 7 Days
3           22  to 28 Days
4            8  to 14 Days
               ...        
17270          2 to 7 Days
17271       22  to 28 Days
17272        8  to 14 Days
17273    More than 28 Days
17274             Same Day
Name: TIME_BETWEEN_BOOK_AND_APPT, Length: 17275, dtype: object


AttributeError: 'NoneType' object has no attribute 'unique'

In [28]:
# Dropping un-necessary columns
snee_appointments_df_filtered = snee_appointments_df[['SUB_ICB_LOCATION_CODE', "APPOINTMENT_MONTH", "TIME_BETWEEN_BOOK_AND_APPT"]]
snee_appointments_df_filtered

,SUB_ICB_LOCATION_CODE,APPOINTMENT_MONTH,TIME_BETWEEN_BOOK_AND_APPT
0,06L,JUL2021,1 Day
1,06L,JUL2021,15 to 21 Days
2,06L,JUL2021,2 to 7 Days
3,06L,JUL2021,22 to 28 Days
4,06L,JUL2021,8 to 14 Days
...,...,...,...
17270,07K,DEC2023,2 to 7 Days
17271,07K,DEC2023,22 to 28 Days
17272,07K,DEC2023,8 to 14 Days
17273,07K,DEC2023,More than 28 Days


In [29]:
snee_appointments_df_filtered = snee_appointments_df_filtered.loc[snee_appointments_df["TIME_BETWEEN_BOOK_AND_APPT"] == "Same Day"].groupby['SUB_ICB_LOCATION_CODE',"APPOINTMENT_MONTH"].sum()
snee_appointments_df_filtered

TypeError: 'method' object is not subscriptable